In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns   
import plotting
hspts = plotting.get_hotspot_list()

This notebook provides statistics from Table 1 and Supplementary Table 1.

In [91]:
gdf = gpd.read_file("outputs/d2-events-2d-230929_SVI_shapefile.geojson")
gdf = gdf[['FIPS', 'E_TOTPOP']]

In [92]:
df = pd.read_parquet("outputs/d2-events-2d5-230929.parquet") 

In [93]:
# Get yearly sums of exposure per census tract
df = df.reset_index()
df = df.groupby("GEOID").resample("Y", on="time").sum()

In [94]:
df.drop(columns=["GEOID"], inplace=True)

In [95]:
df.head()

wfday  heatday  smoke_pm_non_zero  smoke_pm_gt_five  \
GEOID       time                                                              
04001942600 2006-12-31      0        8                 29                 2   
            2007-12-31      0       11                 19                 2   
            2008-12-31      1        2                 24                 0   
            2009-12-31      0        4                 22                 3   
            2010-12-31      0        6                 18                 0   

                        hw  _hws  hs  hws  ws  hs5  ...  wfday_2d  _hws_2d  \
GEOID       time                                    ...                      
04001942600 2006-12-31   0    37   0    0   0    0  ...         0       54   
            2007-12-31   0    28   2    0   0    1  ...         0       43   
            2008-12-31   0    26   1    0   0    0  ...         2       43   
            2009-12-31   0    25   1    0   0    0  ...         0       37   
            2010-12-31   0    24   0    0   0    0  ...         0       35   

                        hs_2d  hws_2d  ws_2d  hw_2d  smoke_pm_gt_five_2d  \
GEOID       time                                                           
04001942600 2006-12-31      4       0      0      0                    3   
            2007-12-31      3       0      0      0                    3   
            2008-12-31      2       0      0      0                    0   
            2009-12-31      2       0      0      0                    5   
            2010-12-31      0       0      0      0                    0   

                        hs_2d_5  ws_2d_5  hws_2d_5  
GEOID       time                                    
04001942600 2006-12-31        0        0         0  
            2007-12-31        2        0         0  
            2008-12-31        0        0         0  
            2009-12-31        0        0         0  
            2010-12-31        0        0         0  

[5 rows x 24 columns]

In [96]:
hspts = ['wfday', 'heatday', 'smoke_pm_non_zero', 'smoke_pm_gt_five',
       'hw', '_hws', 'hs', 'hws', 'ws', 'hs5', 'hws5', 'ws5', 'heatday_2d',
       'smoke_pm_non_zero_2d', 'wfday_2d', '_hws_2d', 'hs_2d', 'hws_2d',
       'ws_2d', 'hw_2d', 'smoke_pm_gt_five_2d', 'hs_2d_5', 'ws_2d_5',
       'hws_2d_5']

In [97]:
df = df.reset_index()

In [98]:
def get_mean_and_std(groupped_df, hspts):
    """Get mean and standard deviation of a list of hotspots for a groupped dataframe."""
    mean = groupped_df[hspts].mean()
    std = groupped_df[hspts].std()
    geoid_combined = mean.map('{:,.1f}'.format) + " (" + std.map('{:,.1f}'.format) + ")"
    return geoid_combined

# Tract-day exposure

In [99]:
geoid_group = df[["GEOID"]+ hspts].groupby(["GEOID"]).sum().div(15) # 15 years
year_group = df[["time"]+ hspts].groupby(["time"]).sum()

res = pd.DataFrame()
res["avg_exposure_days_per_year"] = get_mean_and_std(year_group, hspts)
res["avg_exposure_days_per_ct"] = get_mean_and_std(geoid_group, hspts)

# Person-day exposure

In [100]:
pdf = gdf.merge(df, left_on="FIPS", right_on="GEOID")

In [101]:
# multiply hspts by E_TOTPOP to get person-day exposure
pdf[hspts] = pdf[hspts].mul(pdf["E_TOTPOP"], axis=0)
pdf.head()

,FIPS,E_TOTPOP,GEOID,time,wfday,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,_hws,...,wfday_2d,_hws_2d,hs_2d,hws_2d,ws_2d,hw_2d,smoke_pm_gt_five_2d,hs_2d_5,ws_2d_5,hws_2d_5
0,04001942600,1711,04001942600,2006-12-31,0,13688,49619,3422,0,63307,...,0,92394,6844,0,0,0,5133,0,0,0
1,04001942600,1711,04001942600,2007-12-31,0,18821,32509,3422,0,47908,...,0,73573,5133,0,0,0,5133,3422,0,0
2,04001942600,1711,04001942600,2008-12-31,1711,3422,41064,0,0,44486,...,3422,73573,3422,0,0,0,0,0,0,0
3,04001942600,1711,04001942600,2009-12-31,0,6844,37642,5133,0,42775,...,0,63307,3422,0,0,0,8555,0,0,0
4,04001942600,1711,04001942600,2010-12-31,0,10266,30798,0,0,41064,...,0,59885,0,0,0,0,0,0,0,0


In [102]:
geoid_group = pdf[["GEOID"]+ hspts].groupby(["GEOID"]).sum().div(15) # 15 years
year_group = pdf[["time"]+ hspts].groupby(["time"]).sum()

res["avg_exposure_person_days_per_year"] = get_mean_and_std(year_group, hspts)
res["avg_exposure_person_days_per_ct"] = get_mean_and_std(geoid_group, hspts)

In [104]:
res

,avg_exposure_days_per_year,avg_exposure_days_per_ct,avg_exposure_person_days_per_year,avg_exposure_person_days_per_ct
wfday,"7,708.8 (1,218.7)",0.4 (2.5),"25,236,989.4 (4,118,723.7)","1,393.8 (7,850.8)"
heatday,"133,731.9 (47,405.9)",7.4 (3.4),"564,409,493.3 (201,503,687.2)","31,170.8 (19,860.5)"
smoke_pm_non_zero,"481,611.2 (259,849.3)",26.6 (10.7),"1,976,248,549.1 (1,087,496,056.8)","109,142.8 (60,132.5)"
smoke_pm_gt_five,"161,296.6 (168,009.0)",8.9 (5.0),"661,427,020.7 (700,920,429.9)","36,528.8 (24,946.1)"
hw,208.9 (107.3),0.0 (0.1),"699,179.5 (359,093.7)",38.6 (364.2)
_hws,"581,867.0 (268,407.7)",32.1 (10.4),"2,397,675,834.7 (1,122,384,576.9)","132,417.1 (65,985.9)"
hs,"38,217.5 (30,682.0)",2.1 (1.1),"159,114,734.8 (129,949,305.7)","8,787.5 (5,770.7)"
hws,154.2 (98.8),0.0 (0.1),"511,903.5 (330,385.5)",28.3 (289.9)
ws,"2,912.8 (983.5)",0.2 (1.1),"8,917,186.3 (3,074,409.0)","492.5 (3,371.5)"
hs5,"14,663.7 (16,561.7)",0.8 (0.5),"60,863,131.0 (70,262,808.8)","3,361.3 (2,439.6)"
